# Heuristic Candidate Mining - 페이지 필터링

메트릭별로 관련 키워드가 있는 페이지만 추출하여 주석 작업 범위를 줄입니다.

## 설정 및 임포트

In [1]:
import json
import sys
from pathlib import Path

# ========================================
# 프로젝트 경로 설정 (여기만 수정하세요!)
# ========================================
# PROJECT_DIR = Path('../projects/samsung_2024') 
# PROJECT_DIR = Path('../projects/sk_hynix_2024')  
PROJECT_DIR = Path('../projects/hyundai_2024')  

# 파일 경로
METRIC_FILE = PROJECT_DIR / 'metric_sid_map.json'
PAGES_FILE = PROJECT_DIR / 'pages' / 'metadata.json'
OUTPUT_FILE = PROJECT_DIR / 'metric_page_mapping.json'

print(f"✅ 프로젝트: {PROJECT_DIR.name}")
print(f"   메트릭 파일: {METRIC_FILE.exists()}")
print(f"   페이지 파일: {PAGES_FILE.exists()}")

✅ 프로젝트: hyundai_2024
   메트릭 파일: True
   페이지 파일: True


## 데이터 로드

In [2]:
# 메트릭 정의 로드
with open(METRIC_FILE, 'r', encoding='utf-8') as f:
    metrics = json.load(f)

# 페이지 메타데이터 로드
with open(PAGES_FILE, 'r', encoding='utf-8') as f:
    pages = json.load(f)

# 키워드 매핑 로드
sys.path.insert(0, str(PROJECT_DIR))
import importlib
if 'metric_keywords' in sys.modules:
    import metric_keywords
    importlib.reload(metric_keywords)
from metric_keywords import METRIC_KEYWORDS

print(f"✅ 메트릭: {len(metrics)}개")
print(f"✅ 페이지: {len(pages)}개 (페이지 {pages[0]['page']} ~ {pages[-1]['page']})")
print(f"✅ 키워드 매핑: {len(METRIC_KEYWORDS)}개 메트릭")
print(f"\n메트릭 목록:")
for metric_id, info in metrics.items():
    print(f"  {metric_id}: {info['topic']}")

✅ 메트릭: 12개
✅ 페이지: 125개 (페이지 1 ~ 125)
✅ 키워드 매핑: 12개 메트릭

메트릭 목록:
  TR-AU-250a.1: Product Safety
  TR-AU-250a.2: Product Safety
  TR-AU-250a.3: Product Safety
  TR-AU-310a.1: Labour Practices
  TR-AU-310a.2: Labour Practices
  TR-AU-410a.1: Fuel Economy & Use-phase Emissions
  TR-AU-410a.2: Fuel Economy & Use-phase Emissions
  TR-AU-410a.3: Fuel Economy & Use-phase Emissions
  TR-AU-440a.1: Materials Sourcing
  TR-AU-440b.1: Materials Efficiency & Recycling
  TR-AU-440b.2: Materials Efficiency & Recycling
  TR-AU-440b.3: Materials Efficiency & Recycling


In [3]:
# 메트릭별 관련 페이지 추출
metric_page_mapping = {}

for metric_id, keywords in METRIC_KEYWORDS.items():
    related_pages = []
    
    for page_data in pages:
        page_num = page_data['page']
        page_text = page_data.get('text', '').lower()
        
        # 키워드 검색 (대소문자 구분 없음)
        for keyword in keywords:
            if keyword.lower() in page_text:
                related_pages.append(page_num)
                break  # 하나라도 발견되면 다음 페이지로
    
    metric_page_mapping[metric_id] = sorted(set(related_pages))
    
    # 메트릭별 결과 출력
    metric_info = metrics.get(metric_id, {})
    topic = metric_info.get('topic', metric_id)
    print(f"✅ {metric_id} ({topic}): {len(related_pages)}개 페이지")
    if related_pages:
        # 처음 10개만 표시
        preview = related_pages[:10]
        if len(related_pages) > 10:
            print(f"   페이지: {preview} ... (총 {len(related_pages)}개)")
        else:
            print(f"   페이지: {related_pages}")
    else:
        print(f"   ⚠️  관련 페이지를 찾지 못했습니다.")

print(f"\n{'='*60}")
print(f"✅ 페이지 필터링 완료!")
print(f"{'='*60}")

✅ TR-AU-250a.1 (Product Safety): 65개 페이지
   페이지: [4, 5, 6, 7, 11, 15, 16, 17, 19, 20] ... (총 65개)
✅ TR-AU-250a.2 (Product Safety): 62개 페이지
   페이지: [3, 4, 5, 7, 9, 11, 12, 16, 20, 23] ... (총 62개)
✅ TR-AU-250a.3 (Product Safety): 28개 페이지
   페이지: [16, 17, 23, 28, 30, 36, 37, 39, 40, 41] ... (총 28개)
✅ TR-AU-310a.1 (Labour Practices): 30개 페이지
   페이지: [12, 13, 14, 16, 50, 51, 52, 53, 55, 57] ... (총 30개)
✅ TR-AU-310a.2 (Labour Practices): 9개 페이지
   페이지: [10, 22, 35, 60, 102, 105, 111, 114, 116]
✅ TR-AU-410a.1 (Fuel Economy & Use-phase Emissions): 73개 페이지
   페이지: [2, 3, 8, 9, 10, 11, 12, 13, 15, 16] ... (총 73개)
✅ TR-AU-410a.2 (Fuel Economy & Use-phase Emissions): 65개 페이지
   페이지: [3, 5, 6, 7, 9, 10, 11, 12, 13, 15] ... (총 65개)
✅ TR-AU-410a.3 (Fuel Economy & Use-phase Emissions): 88개 페이지
   페이지: [2, 4, 9, 11, 12, 13, 14, 15, 16, 17] ... (총 88개)
✅ TR-AU-440a.1 (Materials Sourcing): 69개 페이지
   페이지: [2, 3, 7, 8, 9, 10, 11, 12, 13, 14] ... (총 69개)
✅ TR-AU-440b.1 (Materials Efficiency & Recycling): 1

## 페이지 필터링 실행

## 5. 메트릭별 관련 페이지 추출

## 6. 결과 요약

## 7. JSON 파일로 저장

## 8. UI에서 사용할 수 있는 형태로 출력

In [ ]:
## 통계 및 저장

In [4]:
# 통계 계산
total_pages = len(pages)
all_related_pages = set()
for pages_list in metric_page_mapping.values():
    all_related_pages.update(pages_list)

excluded_pages = sorted(set(range(1, total_pages + 1)) - all_related_pages)

# 결과 저장
output_data = {
    "metric_page_mapping": metric_page_mapping,
    "summary": {
        "total_pages": total_pages,
        "related_pages": len(all_related_pages),
        "excluded_pages": len(excluded_pages),
        "efficiency": f"{len(all_related_pages)/total_pages*100:.1f}%"
    }
}

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

# 결과 출력
print("\n📊 결과 요약")
print("="*60)
print(f"전체 페이지: {total_pages}개")
print(f"관련 페이지: {len(all_related_pages)}개")
print(f"제외 페이지: {len(excluded_pages)}개")
print(f"작업 효율: {len(all_related_pages)/total_pages*100:.1f}%")
print(f"\n✅ 저장 완료: {OUTPUT_FILE}")
print("="*60)


📊 결과 요약
전체 페이지: 125개
관련 페이지: 124개
제외 페이지: 1개
작업 효율: 99.2%

✅ 저장 완료: ../projects/hyundai_2024/metric_page_mapping.json
